## Кодирование категориальных признаков

    sklearn.preprocessing
    
        MultiLabelBinarizer
        LabelBinarizer
        OneHotEncoder
        LabelEncoder
        OrdinalEncoder
    

In [22]:
import  numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


In [6]:
df = pd.DataFrame({'col1': [10,20,30,40,50], 
                   'col2': ['male','female','female', 'male','male'], 
                   'col3': ['c1','c2','c2','c3','c4']})
df.head()

,col1,col2,col3
0,10,male,c1
1,20,female,c2
2,30,female,c2
3,40,male,c3
4,50,male,c4


### LabelEncoder

    собирает все варианты, нумерует и заменяет на номера

In [59]:
# ничего фантастического не делает - считает различные метки классов, выдет номера и пронумеровывает.
# на выходе - список пронумерованных значений

encoder = LabelEncoder()
encoded = encoder.fit_transform(df['col3'])
print(encoded)
print(encoder.classes_)

# энкодер помнит как и что он кодировал
print(encoder.inverse_transform([1]))

# мы создаем всего одну колонку, поэтому название не из encoder.classes_
df_le = df.join(pd.DataFrame(encoded, index=df.index, columns=['label_encoded']))
df_le.head()

[0 1 1 2 3]
['c1' 'c2' 'c3' 'c4']
['c2']


,col1,col2,col3,label_encoded
0,10,male,c1,0
1,20,female,c2,1
2,30,female,c2,1
3,40,male,c3,2
4,50,male,c4,3


### OneHotEncoder

    собирает все варианты, нумерует и выдает one-hot кодировку

In [63]:
# categories=[['c2','c1','c3','c4']] - задает порядок кодирования, можно не задавать
# !!! по умолчанию выдает в сжатом формате, это не то что нужно

encoder = OneHotEncoder(categories=[['c2','c1','c3','c4']], sparse=False)
encoded = encoder.fit_transform(df[['col3']])
print(encoded)
print(encoder.get_feature_names())

print('\nдекодировка:', encoder.inverse_transform([[0., 1., 0., 0.],[1., 0., 0., 0.]]))

df_ohe = df.join(pd.DataFrame(encoded, index=df.index, columns=encoder.get_feature_names()))
df_ohe.pop('col3') # мы ее закодировали, можно убрать
df_ohe.head()

[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
['x0_c2' 'x0_c1' 'x0_c3' 'x0_c4']

декодировка: [['c1']
 ['c2']]


,col1,col2,x0_c2,x0_c1,x0_c3,x0_c4
0,10,male,0.0,1.0,0.0,0.0
1,20,female,1.0,0.0,0.0,0.0
2,30,female,1.0,0.0,0.0,0.0
3,40,male,0.0,0.0,1.0,0.0
4,50,male,0.0,0.0,0.0,1.0


## OrdinalEncoder



## LabelBinarizer

## MultiLabelBinarizer

    когда в столбцах сидят вектора, которые нужно бинаризировать

In [65]:
df = pd.DataFrame({
    'col1': [10,20,30,40,50,60],
    'col2': ['огурцы,помидоры','огурцы, яблоки, бананы','бананы','помидоры,бананы','яблоки','огурцы']              
})
df.head(10)

,col1,col2
0,10,"огурцы,помидоры"
1,20,"огурцы, яблоки, бананы"
2,30,бананы
3,40,"помидоры,бананы"
4,50,яблоки
5,60,огурцы


In [73]:
# хотим раскидать в one-hot но с учетом того что они разбросаны и тп
def separator(text):
    return set(text.split(','))

col = df['col2'].apply(separator)
col

0            {огурцы, помидоры}
1    { яблоки, огурцы,  бананы}
2                      {бананы}
3            {бананы, помидоры}
4                      {яблоки}
5                      {огурцы}
Name: col2, dtype: object

In [76]:
mlb = MultiLabelBinarizer()

res = mlb.fit_transform(col)
print(mlb.classes_)
res

[' бананы' ' яблоки' 'бананы' 'огурцы' 'помидоры' 'яблоки']


array([[0, 0, 0, 1, 1, 0],
       [1, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0]])